# Datenabgleich mit SHAB
Rubriken: https://docs.google.com/spreadsheets/d/1RmSGX_SH5lLK1u0wJ-0Omp7mD_QJu9sW9-TUu3-X3PY/edit?ts=5a5e0db5#gid=0

In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
endpoint = 'http://prod.eshab.shab.ch/api/v1/'

In [3]:
rubric_translation = [
    #{
    #'rubric': 'HR',
    #'trans': 'Handelsregistereintragung'
    #},
    #{
    #'rubric': 'BH',
    #'trans': 'Bekanntmachungen nach Handelsregisterverordnung'
    #},
    {
    'rubric': 'LS',
    'trans': 'Liquidationsschuldenrufe'
    },
    {
    'rubric': 'SR',
    'trans': 'Weitere gesellschaftsrechtliche Schuldenrufe'
    },
    {
    'rubric': 'SB',
    'trans': 'Schuldbetreibungen'
    },
    {
    'rubric': 'KK',
    'trans': 'Konkurse'
    },
    {
    'rubric': 'UV',
    'trans': 'Gerichtliche Urteile, Verfügungen und Vorladungen'
    }
    
]

In [4]:
#Import csv
df = pd.read_csv('../../data/99. analyzes/csv/2017_hcp_recipients_aggregated.csv')
df_plz = pd.read_csv('../6. database export/sources/Post_Adressdaten20190122_edited.csv')

In [5]:
#Drop
df_plz.drop(['REC_ART', 'ONRP', 'BFSNR', 'PLZ_TYP', 'POSTLEITZAHL', 'PLZ_ZZ', 'SPRACHCODE', 'SPRACHCODE_ABW', 'ORTBEZ27', 'GPLZ', 'BRIEFZ_DURCH', 'GILT_AB_DAT', 'PLZ_BRIEFZUST', 'PLZ_COFF'], axis=1, inplace=True)
df_plz.drop_duplicates(['ORTBEZ18'], inplace=True)

#Lowercase location
df['location_lower'] = df['location'].str.lower()
df_plz['ORTBEZ18'] = df_plz['ORTBEZ18'].str.lower()

#Join recipient and zip
df_joined = df.merge(df_plz, how='left', left_on='location_lower', right_on='ORTBEZ18')

df_joined = df_joined[df_joined.KANTON.notna()]

In [14]:
df_joined = df_joined.sort_values('total', ascending=False)
df_top = df_joined.iloc[6000:7000]

In [15]:
#Translate rubric
def update_rubric(s):
    for rub in rubric_translation:
        if rub['rubric'] == s:
            return rub['trans']
            break

    return s


In [16]:
#create empty ds
col_names =  [
              'name',
              'location',
              'address',
              'shab_rubric',
              'shab_subrubric',
              'shab_title',
              'shab_id'
            ]

df_shab  = pd.DataFrame(columns = col_names)


url = endpoint + "publications/xml?"

for index, row in df_top.iterrows():
    print("=== %s (%s) ===" % (row['name'], index))
    #print(url)
    payload = {
        'keyword': row['name'],
        'cantons': row['KANTON'],
        'publicationStates': 'PUBLISHED'
    }
    
    r = requests.get(url + 'rubrics=LS&rubrics=SR&rubrics=SB&rubrics=KK&rubrics=UV', params=payload)
    #print(payload)
    
    """
    file = open('shab.xml', 'w')
    file.write(r.text)
    file.close()
    """
    
    root = ET.fromstring(r.text)

    for child in root.iter('publication'):
        print(child.find('meta/rubric').text)
        df_shab = df_shab.append({
                        'name': row['name'],
                        'location': row['location'],
                        'address': row['address'],
                        'shab_rubric': child.find('meta/rubric').text,
                        'shab_subrubric': child.find('meta/subRubric').text,
                        'shab_title': child.find('meta/title/de').text,
                        'shab_id': child.find('meta/id').text},
                        ignore_index=True)
        
    


        
df_shab['url'] = df_shab['shab_id'].apply(lambda x: 'https://shab.ch/#!/search/publications/detail/' + x)
df_shab['shab_rubric'] = df_shab['shab_rubric'].apply(lambda x: update_rubric(x))

df_shab.to_csv('shab_records.csv', sep=';', index=False)
df_shab.head()


=== Karin Zingg Brühlmann (3673) ===
=== Sabine Abele (2176) ===
=== Petra Kästner (1822) ===
=== Marco Gebbers (1016) ===
=== Marino Urbinelli (1040) ===
=== Elke Broos-Könitz (3858) ===
=== David Kabashi (3993) ===
=== Lubica Gujanicic (600) ===
=== Jonathan Gonzalez (3771) ===
=== Johannes Berger (3790) ===


,name,location,address,shab_rubric,shab_subrubric,shab_title,shab_id,url


In [9]:
print("fertig")

fertig


In [ ]:
file = open('shab.xml', 'w')
file.write(r.text)
file.close()

In [ ]:
payload = {
        'keyword': 'Anton Luchsinger',
        'cantons': 'zh',
        'publicationStates': 'PUBLISHED'
    }

r = requests.get(url + 'rubrics=LS&rubrics=SR&rubrics=SB&rubrics=KK&rubrics=UV', params=payload)

In [ ]:
file = open('shab.xml', 'w')
file.write(r.text)
file.close()